In [98]:
import os
import sklearn
import pandas as pd
import csv

In [99]:
n_gram_value = 5

In [196]:
def get_ngrams(input_list, n):    
    return zip(*[input_list[i:] for i in range(n)])


In [197]:
def getAllPossibleWords(file_path): 
    possible_words=[]
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                possible_words.append(word)     
   # possible_words = map(lambda x : unidecode.unidecode(x),possible_words)
    for c in ['`',"[","]","(",")","\"",".","''",","]:
        possible_words = map(lambda x: x.replace(c,""),possible_words)
        
    possible_words = map(lambda x: x.replace('&','And'),possible_words)
    #possible_words = map(lambda x: x.replace('of','Of'),possible_words)
    possible_words=filter(lambda x:len(x)>0,possible_words)    
    
    return possible_words

In [198]:
def getListFromFiles(filename):
    file_path = os.path.join(os.getcwd(),filename)
    info_list = []
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                info_list.append(word)
    return info_list

In [199]:
def writeToCsv(processed_list):
    with open("output-sreejita.csv",'wb') as resultFile:
        wr = csv.writer(resultFile, dialect='excel')
        #for row in processed_list:
        wr.writerows(processed_list)

In [200]:
'''def preprocessing(possible_words,doc_id):    
    suffix_list = getListFromFiles("company-suffixes.txt")
    prefix_list = getListFromFiles("company-prefixes.txt")    
    common_words = getListFromFiles("common-words.txt")
    preprocessed_list =[]    
    marked_list=[]
    word_number=len(possible_words)
    for i in range(5,0,-1):
        word_count=0
        ngram_list = get_ngrams(possible_words,i)
        for ngram in ngram_list:            
            #print ngram   
            
            #if markup is both at the beginning or at the end of the word group accept and label as 1
            if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])
            
            #get all instances of format <markup>Microsoft</markup>
            if(len(ngram)==1 and '</markup>' in ngram[0] and '<markup>' in ngram[0]):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])            
                    
            else:
                #prune away all n grams with non uppercase first character
                if(all(word[0].isupper() and "markup" not in word for word in ngram)):       
                    
                    # prune common words
                    if(any(word not in common_words for word in ngram)):                        
                        preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])

                    #prune unigrams 
                    elif(i==1 and word_number> word_count+1):                        
                        
                        #add the unigram to the list if the next word begins with lower case
                        if(possible_words[word_count+1][0].islower()):
                            preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])  
            word_count = word_count+1
    return preprocessed_list   ''' 

'def preprocessing(possible_words,doc_id):    \n    suffix_list = getListFromFiles("company-suffixes.txt")\n    prefix_list = getListFromFiles("company-prefixes.txt")    \n    common_words = getListFromFiles("common-words.txt")\n    preprocessed_list =[]    \n    marked_list=[]\n    word_number=len(possible_words)\n    for i in range(5,0,-1):\n        word_count=0\n        ngram_list = get_ngrams(possible_words,i)\n        for ngram in ngram_list:            \n            #print ngram   \n            \n            #if markup is both at the beginning or at the end of the word group accept and label as 1\n            if(len(ngram)>1 and \'<markup>\' in ngram[0] and \'</markup>\' in ngram[len(ngram)-1] and\n              all((word[0].isupper() or "markup" in word) for word in ngram) and \n              ["<markup>" in word for word in ngram].count(True)==1):\n                preprocessed_list.append([\' \'.join(ngram),doc_id,word_count,word_count+i-1,1])\n            \n            #get all

In [201]:
def preprocessing(possible_words,doc_id):    
    suffix_list = getListFromFiles("company-suffixes.txt")
    prefix_list = getListFromFiles("company-prefixes.txt")    
    common_words = getListFromFiles("common-words.txt")
    preprocessed_list =[]    
    marked_list=[]
    word_number=len(possible_words)
    for i in range(5,0,-1):
        word_count=0
        ngram_list = get_ngrams(possible_words,i)
        for ngram in ngram_list:            
            #print ngram   
            
            #if markup is both at the beginning or at the end of the word group accept and label as 1
            if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1): 
                company_tuple = ngram
                for string in ["<markup>","</markup>"]:
                    company_tuple = map(lambda x : x.replace(string,""),company_tuple)                
                preprocessed_list.append([' '.join(company_tuple),doc_id,word_count,word_count+i-1,1])
            
            #get all instances of format <markup>Microsoft</markup>
            elif(len(ngram)==1 and '</markup>' in ngram[0] and '<markup>' in ngram[0]):
                
                company_tuple = ngram
                for string in ["<markup>","</markup>"]:
                    company_tuple = map(lambda x : x.replace(string,""),company_tuple) 
                
                preprocessed_list.append([' '.join(company_tuple),doc_id,word_count,word_count+i-1,1])            
                    
            else:
                #prune away all n grams with non uppercase first character
                if(all(word[0].isupper() and "markup" not in word for word in ngram)):  
                    
                    #prune unigrams 
                    if(i==1):                        
                        if(ngram[0] not in suffix_list):
                            if(word_number> word_count+1 and ngram[0] not in common_words):
                                #add the unigram to the list if the next word begins with lower case
                                if(possible_words[word_count+1][0].islower()):
                                    preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])
                    
                    #prune common words
                    elif(any(word not in common_words for word in ngram)):                        
                        preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])                   

                    
                     
            word_count = word_count+1
    return preprocessed_list    

In [202]:
def generateFeatures(filename, processed_list,possible_words):
    
    company_suffixes = getListFromFiles("company-suffixes.txt")
    company_prefixes = getListFromFiles("company-prefixes.txt")    
    company_list=[]
    final_processed_list = []
    splittedFilename = filename.split("-")
    i=0
    print "possible_words- "  
    print possible_words
    for company in processed_list:
        i+=1
        #Feature 1 : name has any prefix or suffix of a company 
        #if company name has any one of the suffixes or prefixes set it to true 
        hasCompanyid=0        
        if(company[0].split()[-1] in company_suffixes or company[0].split()[0] in company_prefixes):
            hasCompanyid = 1
            company_list.append(company[0])            
        #company.append(hasCompanyid)
        
        
        #has Company name at the beginning ?
        #Feature 2: name occurs at the beginning of a company name
        hasCompanyNameFirst = 0
        l1= [x for x in company[0].split() if x not in company_prefixes]
        l2=[]
        for comp in company_list:    
            if(comp.split()[0] in company_prefixes):
                l2.append(" ".join(comp.split()[1:]))
            else:
                l2.append(comp)
        if(any(word in l2 for word in l1)):
            hasCompanyNameFirst = 1
        elif(any(company[0].split()[0]== word.split()[0] for word in company_list)):
            hasCompanyNameFirst = 1 
        
            
        
        #has Company name as a substring ?
        #Feature 3: name is a substring in the company name
        hasNameSubstring=0
        for comp in company_list:
            l1 = company[0].split()
            l2 = comp.split()
            if(len(set(l1)&set(l2)) > 0 ):
                hasNameSubstring=1       
       
         
        
        #Feature 4: name in filename
        isInFilename = 0
        splittedInstance = company[0].split()
        for sp in splittedInstance:
            if sp.lower() in splittedFilename:
                isInFilename = 1 
        
        
        #Feature 5-y - Check neibhouring words - check 3 words before and 3 words after
        starting_loc = company[2]
        ending_loc = company[3]
        words_before_starting_loc = []
        words_after_ending_loc = []
        j= starting_loc-1
        count = 1
        while j >= 0 and count <= 3:
            words_before_starting_loc.append(possible_words[company[2]-count])
            j-=1
            count+=1
        
        j = ending_loc+1
        count = 1
        while j < len(possible_words) and count <= 3:
            words_after_ending_loc.append(possible_words[company[3]+count])
            j+=1
            count+=1
        
        #Feature 5 - Check for word "based" before the company
        has_based = 0
        for word in words_before_starting_loc:
            if "based" in word.lower():
                has_based = 1
        
        #Feature 6 "'s" is after the company
        has_apostrophe_s = 0
        for word in words_after_ending_loc:
            if "'s" in word.lower():
                has_apostrophe_s = 1
        '''
        #Feature 7 "," is after/before the company
        has_comma = 0
        for word in words_after_ending_loc:
            if "," in word.lower():
                has_comma = 1
        
        for word in words_before_starting_loc:
            if "," in word.lower():
                has_comma = 1
        '''
        #Feature 8 designations before/after company
        desig = open('designations.txt').read().split('\n')
        desig_lower = [x.lower() for x in desig]
        has_desig = 0
        for word in words_after_ending_loc:
            if word.lower() in desig_lower:
                has_desig = 1
              
        for word in words_before_starting_loc:
            if word.lower() in desig_lower:
                has_desig = 1 
         
        #Feature 9 "said/told" in the before company
        has_said = 0
        for word in words_before_starting_loc:
            if "said" in word.lower() or "told" in word.lower():
                has_said = 1
        
        '''
        #Feature 10 Prune away Examples if previous/next word starts with a Capital
        surrounded_capital = 0
        if len(words_before_starting_loc) > 0:
            prev_word = words_before_starting_loc[0]
            if len(prev_word) > 0 and prev_word[0].isupper():
                surrounded_capital = 1
        if len( words_after_ending_loc) > 0:
            next_word = words_after_ending_loc[0]
            if len(next_word) > 0 and next_word[0].isupper():
                surrounded_capital = 1
        '''
          
                
        
        print "company:- " + company[0]
        print "words_after_ending_loc"
        print words_after_ending_loc
        print "words_before_starting_loc"
        print words_before_starting_loc
        
        company.extend([hasCompanyid, hasCompanyNameFirst, hasNameSubstring, isInFilename, has_based, has_apostrophe_s, has_desig, has_said])
        
            

In [203]:
base_directory = os.path.dirname(os.getcwd())
data_directory = os.path.join(base_directory,"Dataset","Training")
file_list = os.listdir(data_directory)
possible_words=[]
processed_list=[]
count=0
#doc_id = 101
for file in file_list:
    relevant_filename = file[4:-4]
    doc_id = file[0:3]
    #print(doc_id)
    #if(int(doc_id)!=95):
    #    continue    
    possible_words = getAllPossibleWords(os.path.join(data_directory,file))     
    
    #start from here .. processessed list has the structure[[word,doc_id,word_count_start,word_count_end,label]]
    ''''processed_list.append(preprocessing(possible_words,doc_id))
    writeToCsv(processed_list)'''
    processed_list_for_doc = preprocessing(possible_words,doc_id)
    #print processed_list_for_doc
    generateFeatures(relevant_filename, processed_list_for_doc,possible_words)
    processed_list.extend(processed_list_for_doc)
    #writeToCsv(processed_list) 
  

possible_words- 
['American', 'Idol', 'the', 'most', 'popular', 'show', 'in', 'primetime', 'television', 'withstood', 'last', "week's", 'criticism', 'over', 'judging', 'and', 'returned', 'last', 'night', 'with', 'the', 'third-largest', 'audience', 'for', 'an', 'entertainment', 'show', 'this', 'season', 'With', 'about', '326', 'million', 'viewers', 'according', 'to', 'overnight', 'results', 'from', 'Nielsen', 'Media', 'Research', 'the', 'latest', 'Idol', 'trailed', 'only', 'the', 'two', 'Idol', 'shows', 'that', 'opened', 'its', 'sixth', 'season', 'on', 'Fox', 'last', 'week', 'with', 'audiences', 'of', '374', 'million', 'and', '37', 'million', 'and', 'the', 'National', 'Football', "League's", 'American', 'Conference', 'championship', 'game', 'on', 'CBS', 'last', 'Sunday', 'which', 'drew', '435', 'million', 'The', 'first', 'two', 'Idol', 'shows', 'were', 'the', 'second', 'and', 'third', 'most-watched', 'of', 'the', 'series', 'since', 'the', "show's", '2002', 'debut', 'They', 'also', 'spur

words_before_starting_loc
['ask', 'would', 'it']
company:- Spokespersons
words_after_ending_loc
['for', '<markup>Advanced', 'Marketing</markup>']
words_before_starting_loc
['15', 'Feb', 'held']
possible_words- 
['<markup>OAO', 'Aeroflot</markup>', 'AFLT', 'eastern', 'Europe', "'s", 'largest', 'airline', 'carried', '87', 'percent', 'more', 'passengers', 'last', 'year', 'as', 'the', "country's", 'growing', 'economy', 'boosted', 'travel', 'State-controlled', '<markup>Aeroflot</markup>', 'flew', '73', 'million', 'passengers', 'in', '2006', 'the', 'Moscow-based', 'airline', 'said', 'today', 'in', 'a', 'statement', 'The', 'company', 'handled', '02', 'percent', 'less', 'cargo', 'or', '145300', 'tons', 'last', 'year', 'The', 'load', 'factor', 'or', 'proportion', 'of', 'seats', 'filled', 'increased', 'to', '701', 'percent', 'from', '691', 'percent', 'in', '2005', '<markup>Aeroflot</markup>', 'is', 'attracting', 'more', 'travelers', 'as', 'Russia', "'s", 'economy', 'expands', 'for', 'a', 'ninth'

company:- Nasdaq Stock
words_after_ending_loc
['Market', 'composite', 'trading']
words_before_starting_loc
['in', 'pm', '4']
company:- Stock Market
words_after_ending_loc
['composite', 'trading', 'and']
words_before_starting_loc
['Nasdaq', 'in', 'pm']
company:- Colorado Iowa
words_after_ending_loc
['Missouri', 'Mississippi', 'and']
words_before_starting_loc
['in', 'casinos', 'operates']
company:- Iowa Missouri
words_after_ending_loc
['Mississippi', 'and', 'Nevada']
words_before_starting_loc
['Colorado', 'in', 'casinos']
company:- Missouri Mississippi
words_after_ending_loc
['and', 'Nevada', 'Neilsen']
words_before_starting_loc
['Iowa', 'Colorado', 'in']
company:- Nevada Neilsen
words_after_ending_loc
['owned', '55', 'percent']
words_before_starting_loc
['and', 'Mississippi', 'Missouri']
company:- Eric Green
words_after_ending_loc
['director', 'of', 'research']
words_before_starting_loc
['said', 'them', 'liquidate']
company:- Cherry Hills
words_after_ending_loc
['New', 'Jersey', 'You']


possible_words- 
['<markup>Industrial', 'And', 'Commercial', 'Bank', 'of', 'China', 'Ltd</markup>', 'and', 'other', 'Chinese', 'financial', 'stocks', 'plunged', 'on', 'concern', 'the', 'government', 'will', 'clamp', 'down', 'on', 'lending', 'to', 'cool', 'share', 'price', 'gains', 'and', 'corporate', 'investment', '<markup>ICBC</markup>', 'the', "nation's", 'biggest', 'bank', 'fell', '8', 'percent', 'to', '469', 'yuan', '<markup>Citic', 'Securities', 'Co</markup>', 'the', 'largest', 'publicly', 'traded', 'brokerage', 'tumbled', '97', 'percent', 'to', 'close', 'at', '3621', 'yuan', '<markup>China', 'Life', 'Insurance', 'Co</markup>', 'the', "nation's", 'biggest', 'insurer', 'shed', '9', 'percent', 'to', '3389', 'yuan', 'Financial', 'stocks', 'fell', 'along', 'with', 'the', 'wider', 'market', 'which', 'recorded', 'its', 'biggest', 'tumble', 'in', 'a', 'decade', "China's", 'benchmark', 'stock', 'index', 'rose', 'to', 'a', 'record', 'yesterday', 'and', 'banking', 'shares', 'have', 'climbed

company:- Index
words_after_ending_loc
['have', 'jumped', '63']
words_before_starting_loc
['Composite', 'Shenzhen', 'the']
company:- Asia-Pacific
words_after_ending_loc
['region', 'The', 'measures']
words_before_starting_loc
['the', 'in', 'benchmarks']
company:- Composite
words_after_ending_loc
['which', 'tracks', 'yuan-denominated']
words_before_starting_loc
['Shanghai', 'The', 'years']
company:- B
words_after_ending_loc
['shares', 'rose', '2967']
words_before_starting_loc
['foreign-currency', 'and', 'shares']
company:- Composite
words_after_ending_loc
['which', 'tracks', 'the']
words_before_starting_loc
['Shenzhen', 'The', 'today']
company:- China's
words_after_ending_loc
['two', 'stock', 'exchanges']
words_before_starting_loc
['of', 'smaller', 'the']
company:- Dutch
words_after_ending_loc
['bank', 'and', '<markup>Tianjin']
words_before_starting_loc
['largest', 'the', 'NV</markup>']
company:- Tianjin
words_after_ending_loc
['government', 'The', 'company']
words_before_starting_loc
['

...

['The', 'based', 'is']
possible_words- 
['<markup>OAO', 'Rosneft</markup>', 'ROSN', "Russia's", 'state', 'oil', 'company', 'may', 'sell', 'shares', 'to', 'pay', 'off', 'the', 'debt', 'it', 'amassed', 'to', 'acquire', 'most', 'of', 'the', 'assets', 'of', 'bankrupt', '<markup>Yukos', 'Oil', 'Co</markup>', 'Chief', 'Executive', 'Officer', 'Sergei', 'Bogdanchikov', 'said', '<markup>Rosneft</markup>', 'bought', '94', 'percent', 'of', 'its', 'own', 'stock', 'at', 'a', '<markup>Yukos</markup>', 'auction', 'this', 'year', 'and', 'plans', 'to', 'swap', 'most', 'of', 'the', 'shares', 'for', 'assets', 'Bogdanchikov', 'said', 'in', 'an', 'interview', 'in', 'Rome', 'today', 'A', 'smaller', 'part', 'may', 'be', 'sold', 'for', 'cash', 'or', 'be', 'used', 'to', 'back', 'a', 'convertible', 'bond', 'sale', 'Bogdanchikov', 'said', 'We', 'may', 'sell', 'a', 'minority', 'of', 'these', 'shares', 'the', 'majority', 'will', 'go', 'to', 'a', 'swap', 'Bogdanchikov', 'said', 'adding', 'that', 'the', 'company', '

company:- New York
words_after_ending_loc
['trading', 'after', '<markup>Banc']
words_before_starting_loc
['in', 'year', 'a']
company:- America Securities
words_after_ending_loc
['LLC</markup>', 'downgraded', 'its']
words_before_starting_loc
['of', '<markup>Banc', 'after']
company:- Nordstrom Inc
words_after_ending_loc
['JWN', 'also', 'fell']
words_before_starting_loc
['shares', 'its', 'downgraded']
company:- Dana Cohen
words_after_ending_loc
['cut', 'her', 'recommendation']
words_before_starting_loc
['analyst', 'America</markup>', 'of']
company:- Great Depression
words_after_ending_loc
['The', 'high', 'end']
words_before_starting_loc
['the', 'since', 'time']
company:- Depression The
words_after_ending_loc
['high', 'end', 'is']
words_before_starting_loc
['Great', 'the', 'since']
company:- New York-based
words_after_ending_loc
['Cohen', 'wrote', 'in']
words_before_starting_loc
['fall', 'to', 'farthest']
company:- York-based Cohen
words_after_ending_loc
['wrote', 'in', 'her']
words_before

['early', 'this', 'month']
words_before_starting_loc
['northern', 'in', 'Yanshan']
company:- Shanghai
words_after_ending_loc
['refinery', 'for', 'two']
words_before_starting_loc
['its', 'in', 'unit']
company:- Beijing-based
words_after_ending_loc
['company', 'said', 'in']
words_before_starting_loc
['the', '20', 'Nov']
company:- Sinopecnews
words_after_ending_loc
['today', 'without', 'elaborating']
words_before_starting_loc
['newsletter', 'online', 'its']
company:- China
words_after_ending_loc
['the', "world's", 'second-largest']
words_before_starting_loc
['elaborating', 'without', 'today']
company:- Some
words_after_ending_loc
['refineries', 'have', 'reduced']
words_before_starting_loc
['demand', 'behind', 'lags']
company:- Sinopec
words_after_ending_loc
['said', 'in', 'the']
words_before_starting_loc
['supply', 'increase', 'and']
company:- All
words_after_ending_loc
['ethylene', 'units', 'are']
words_before_starting_loc
['statement', 'the', 'to']
company:- China's
words_after_ending_l

words_after_ending_loc
['requires', 'airlines', 'to']
words_before_starting_loc
['the', 'held', 'closely']
company:- Richard Branson
words_after_ending_loc
['lost', '$348', 'million']
words_before_starting_loc
['billionaire', 'UK', 'by']
company:- Virgin America
words_after_ending_loc
['had', 'operating', 'revenue']
words_before_starting_loc
['data', 'government', 'US']
company:- Transportation Statistics'
words_after_ending_loc
['Web', 'site', 'In']
words_before_starting_loc
['of', 'Bureau', 'the']
company:- Statistics' Web
words_after_ending_loc
['site', 'In', 'the']
words_before_starting_loc
['Transportation', 'of', 'Bureau']
company:- Abby Lunardini
words_after_ending_loc
['said', 'in', 'an']
words_before_starting_loc
['spokeswoman', 'airline', 'overhead']
company:- Virgin America
words_after_ending_loc
['based', 'in', 'Burlingame']
words_before_starting_loc
['model', 'business', 'our']
company:- Burlingame California
words_after_ending_loc
['began', 'flights', 'Aug']
words_before_

['said', 'in', 'the']
words_before_starting_loc
['period', 'trading', 'Christmas']
company:- Chief Executive Officer Richard
words_after_ending_loc
['Goyder', 'said', 'in']
words_before_starting_loc
['period', 'trading', 'Christmas']
company:- Executive Officer Richard Goyder
words_after_ending_loc
['said', 'in', 'the']
words_before_starting_loc
['Chief', 'period', 'trading']
company:- Coles Group Ltd
words_after_ending_loc
['Australia', "'s", 'second-largest']
words_before_starting_loc
['of', 'acquisition', 'billion']
company:- Chief Executive Officer
words_after_ending_loc
['Richard', 'Goyder', 'said']
words_before_starting_loc
['period', 'trading', 'Christmas']
company:- Executive Officer Richard
words_after_ending_loc
['Goyder', 'said', 'in']
words_before_starting_loc
['Chief', 'period', 'trading']
company:- Officer Richard Goyder
words_after_ending_loc
['said', 'in', 'the']
words_before_starting_loc
['Executive', 'Chief', 'period']
company:- Wesfarmers Ltd
words_after_ending_loc
[

In [204]:
ngram = ('<markup>Gazprom', 'Lukoil</markup>')
if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
    print "hallo"


hallo


In [205]:
["markup" in word for word in ngram].count(True)==2

True

In [206]:
import pycountry
country_names = []
for country in pycountry.countries:
    country_names.append(country.name)
    if hasattr(country,'common_name'):
        country_names.append(country.common_name)
    if hasattr(country, 'official_name'):
        country_names.append(country.official_name)

country_names.append("America")
country_names.append("Korea")
country_names.append("South Korea")
country_names.append("North Korea")
country_names.append("U.S.")
country_names.append("Russia")
country_names.append("Siberia")

if "Russia" in country_names:
    print "YES"
    
continent_list = ['Asia', 'Africa', 'South America', 'North America', 'Antarctica', 'Europe', 'Australia']

YES


In [207]:
cities = []
with open('world-cities.csv', 'r') as inp:
        for row in csv.reader(inp):
            if(row and row[3] != "" and row[0]!="name"):
                cities.append(row[0])

cities.append('Bombay')
cities.append('New York')
if "Compton" in cities:
    print "YES"

us_states = open('us-states.txt').read().split('\n')
ethni_list = open('ethnicities.txt').read().split('\n')

YES


In [208]:
desig = open('designations.txt').read().split('\n')
desig_lower = [x.lower() for x in desig]

print desig_lower

['chief executive officer', 'chief financial officer', 'co-chief executive', 'chief investment officer', 'analyst', 'ceo', 'cio', 'cto', 'cfo', 'manager', 'director', 'president', 'vice-president', 'vice president', 'chief', 'executive', 'officer', 'officers', 'analysts', 'managers', 'directors', 'co-chief', 'co chief', 'commissioner', 'minister', 'magistrate', 'judge', 'chairman']


In [209]:
pruned_pos = open('pruned_pos.txt').read().split('\n')
stock_x = open('stock-exchange-list.txt').read().split('\n')
unigram_prune = open('unigram-prune-neg.txt').read().split('\n')
special_char = [';', ',', ':', '?']
prune = open('pruning-neg.txt').read().split('\n') 
prune_lower = [x.lower() for x in prune]
suffixes = open('company-suffixes.txt').read().split('\n')
prefixes = open('company-prefixes.txt').read().split('\n')
rows = 0
print len(processed_list)
for doc in processed_list:
    rows += len(doc)


8120


In [210]:
print pruned_pos
new_proc_list = []
pruned_list = []

for inst in processed_list:
    if inst[0] not in pruned_pos: 
        if inst[0] in country_names or inst[0] in continent_list or inst[0] in cities or inst[0] in stock_x or inst[0] in us_states:
            pruned_list.append(inst)
        elif "'s" in inst[0]:
            pruned_list.append(inst)
        elif inst[0].endswith('n') and inst[0][:-1] in country_names or str in continent_list:
            pruned_list.append(inst)
        elif "based" in inst[0]:
            pruned_list.append(inst)
        elif any(d in inst[0].lower().split() for d in desig_lower):
            pruned_list.append(inst)
        elif any(p in inst[0].lower().split() for p in prune_lower):
            pruned_list.append(inst)
        elif '(' in inst[0] or ')' in inst[0]:
            pruned_list.append(inst)
        elif any(c in inst[0] for c in special_char):
            pruned_list.append(inst)
        elif len(inst[0].split()) == 1:
            if len(inst[0]) == 1:
                pruned_list.append(inst)
            elif inst[0] in unigram_prune or inst[0] in ethni_list:
                pruned_list.append(inst)
            elif inst[0] == inst[0].upper():
                pruned_list.append(inst)
            elif inst[0] in suffixes or inst[0] in prefixes:
                pruned_list.append(inst)
            else:
                new_proc_list.append(inst)
        else:
            new_proc_list.append(inst)
    else:
        new_proc_list.append(inst)
            
                
print len(pruned_list)
for eg in pruned_list:
    if eg[4] == 1:
        print "HOLY MOLY!"
#writeToCsv(processed_list)
print len(new_proc_list)
print len(pruned_list) + len(new_proc_list)
#print pruned_list



['ACE', "Sam's Club", 'ICBC', 'KKR', 'CSN', 'Tata', 'Wheeling', "River's Edge Pharmaceuticals LLC", "River's Edge", 'RWE', 'Fremont', 'News Corp', 'GM', 'Liberty', 'Financial Services Authority', 'FSA', 'NZOG', 'Norilsk', 'NTP', 'MPC', 'PPR', 'THQ', 'Nokia', 'RCS', 'Intercontinental Exchange Inc', 'EON', 'RHB', 'WCM', 'Macclesfield', 'Carrefour', 'KPI', 'TXU', 'ECI', 'BP', 'TNK-BP', 'Imperial', 'Zug', 'TDF', 'TMK', "Ivernia Inc's", 'SAIC', 'GE', 'PICC', 'Saku', 'Winterthur', 'ZKB', "Rupert Murdoch's News Corp", 'PIK', 'VTB', 'UBS', 'IGT', 'BHP', 'BBH', 'China Banking Regulatory Commission', 'Compton', 'Centennial', 'NPD', 'HSBC', "Victoria's Secret", 'MAN', 'MBIA', 'Freeport', 'MGIC', 'PMI', 'Stillwater', "Dr Reddy's Laboratories Ltd", "Royal Mail's", 'Cia Siderurgica Nacional SA', 'Cia Siderurgica Nacional', 'Vector Hospitality Plc', 'Vector', 'Victory']
3543
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY M

In [211]:
'''thefile = open('pruned_pos.txt', 'w')
for item in pruned_pos:
    #print item
    thefile.write("%s\n" % item)
len(pruned_pos)
print pruned_list'''

'thefile = open(\'pruned_pos.txt\', \'w\')\nfor item in pruned_pos:\n    #print item\n    thefile.write("%s\n" % item)\nlen(pruned_pos)\nprint pruned_list'

In [212]:
with open("train_pruned.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in new_proc_list:
        writer.writerow(val)